In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Mediapipe 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 손 인식 설정 (두 손 인식)
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# OpenCV 윈도우 설정
cv2.namedWindow('Driving Simulator', cv2.WINDOW_NORMAL)

# 시뮬레이터 변수 초기화
gear = 'P'  # P, N, D
speed = 0
max_speed = 100
acceleration = 50
deceleration = 30

# UI 요소 위치 및 크기
gear_shifter_pos = None
accelerator_pos = None
brake_pos = None
steering_wheel_pos = None
signal_left = False
signal_right = False
signal_left_on = False
signal_right_on = False
wiper_on = False  # 와이퍼 상태
rain_intensity = 0  # 비 강도

# 신호 타이머
signal_timer = 0
signal_interval = 0.5  # 신호 간격 (초)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

def draw_gear_shifter(frame, gear):
    """기어 시프터 그리기"""
    global gear_shifter_pos
    gear_shifter_pos = (frame.shape[1] - 250, 100)  # 위치 조정
    gears = ['P', 'N', 'D']
    shifter_height = 300  # 높이 증가
    
    cv2.rectangle(frame, (gear_shifter_pos[0] - 40, gear_shifter_pos[1]),
                  (gear_shifter_pos[0] + 40, gear_shifter_pos[1] + shifter_height), (255, 255, 255), 2)
    
    for i, g in enumerate(gears):
        y = gear_shifter_pos[1] + i * (shifter_height // 3) + (shifter_height // 6)
        cv2.putText(frame, g, (gear_shifter_pos[0] - 70, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    shifter_pos = gear_shifter_pos[1] + ['P', 'N', 'D'].index(gear) * (shifter_height // 3) + (shifter_height // 6)
    cv2.circle(frame, (gear_shifter_pos[0], shifter_pos), 20, (0, 255, 0), -1)

def draw_pedals(frame, accelerator_pressure, brake_pressure):
    """가속 페달과 브레이크 페달 그리기"""
    global accelerator_pos, brake_pos
    pedal_width = 80
    pedal_height = 160
    
    accelerator_pos = (frame.shape[1] - 220, frame.shape[0] - 200)
    brake_pos = (frame.shape[1] - 120, frame.shape[0] - 200)
    
    # 가속 페달
    cv2.rectangle(frame, accelerator_pos, (accelerator_pos[0] + pedal_width, accelerator_pos[1] + pedal_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (accelerator_pos[0], accelerator_pos[1] + int((1 - accelerator_pressure) * pedal_height)),
                  (accelerator_pos[0] + pedal_width, accelerator_pos[1] + pedal_height), (0, 255, 0), -1)
    cv2.putText(frame, "Accel", (accelerator_pos[0], accelerator_pos[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # 브레이크 페달
    cv2.rectangle(frame, brake_pos, (brake_pos[0] + pedal_width, brake_pos[1] + pedal_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (brake_pos[0], brake_pos[1] + int((1 - brake_pressure) * pedal_height)),
                  (brake_pos[0] + pedal_width, brake_pos[1] + pedal_height), (0, 0, 255), -1)
    cv2.putText(frame, "Brake", (brake_pos[0], brake_pos[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_steering_wheel(frame, angle):
    """스티어링 휠 그리기"""
    global steering_wheel_pos
    steering_wheel_pos = (150, frame.shape[0] - 150)  # 왼쪽 하단으로 위치 조정
    wheel_radius = 150  # 스티어링 휠 크기 증가
    
    # 스티어링 휠 원 그리기
    cv2.circle(frame, steering_wheel_pos, wheel_radius, (255, 255, 255), 2)
    
    # 회전 행렬을 사용하여 핸들을 회전
    angle_rad = np.deg2rad(angle)
    handle_length = wheel_radius
    handle_end = (int(steering_wheel_pos[0] + handle_length * np.cos(angle_rad)),
                  int(steering_wheel_pos[1] - handle_length * np.sin(angle_rad)))
    cv2.line(frame, steering_wheel_pos, handle_end, (0, 255, 0), 2)
    
    # 핸들 표시
    cv2.putText(frame, f'Steering: {angle}°', (steering_wheel_pos[0] - 50, steering_wheel_pos[1] + wheel_radius + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_signals(frame):
    """좌우 신호등 그리기"""
    signal_radius = 20
    signal_spacing = 60
    
    # 왼쪽 신호
    left_signal_pos = (steering_wheel_pos[0] - signal_spacing, steering_wheel_pos[1])
    cv2.circle(frame, left_signal_pos, signal_radius, (0, 255, 255) if signal_left_on and signal_left else (100, 100, 100), -1)
    
    # 오른쪽 신호
    right_signal_pos = (steering_wheel_pos[0] + signal_spacing, steering_wheel_pos[1])
    cv2.circle(frame, right_signal_pos, signal_radius, (0, 255, 255) if signal_right_on and signal_right else (100, 100, 100), -1)

def draw_wiper(frame, wiper_on):
    """와이퍼 그리기"""
    global steering_wheel_pos
    wiper_length = 100
    wiper_width = 10
    wiper_color = (0, 255, 255) if wiper_on else (100, 100, 100)
    
    # 와이퍼 기둥
    wiper_pivot = (steering_wheel_pos[0], steering_wheel_pos[1] - 100)
    cv2.line(frame, wiper_pivot, (wiper_pivot[0], wiper_pivot[1] - wiper_length), wiper_color, wiper_width)
    
    # 와이퍼 팔
    wiper_angle_rad = np.deg2rad(45)
    wiper_end = (int(wiper_pivot[0] + wiper_length * np.cos(wiper_angle_rad)),
                 int(wiper_pivot[1] - wiper_length * np.sin(wiper_angle_rad)))
    cv2.line(frame, wiper_pivot, wiper_end, wiper_color, wiper_width)

def draw_speedometer(frame, speed, max_speed):
    """속도계 그리기"""
    speedometer_pos = (150, 150)  # 속도계의 중심 위치
    speedometer_radius = 100  # 속도계의 반지름
    cv2.circle(frame, speedometer_pos, speedometer_radius, (255, 255, 255), 2)
    
    # 속도계 바늘
    angle = np.deg2rad((speed / max_speed) * 270 - 135)
    needle_end = (int(speedometer_pos[0] + speedometer_radius * np.cos(angle)),
                  int(speedometer_pos[1] - speedometer_radius * np.sin(angle)))
    cv2.line(frame, speedometer_pos, needle_end, (0, 255, 0), 2)
    
    # 속도 표시
    cv2.putText(frame, f'Speed: {int(speed)} km/h', (speedometer_pos[0] - 50, speedometer_pos[1] + speedometer_radius + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def add_rain_effect(frame, rain_intensity):
    """비 효과 추가"""
    rain_layer = np.random.randint(0, 256, frame.shape, dtype=np.uint8)
    rain_layer = cv2.GaussianBlur(rain_layer, (0, 0), 1.0)
    rain_layer = cv2.addWeighted(rain_layer, rain_intensity, frame, 1 - rain_intensity, 0)
    return rain_layer

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 손 추적
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    hand_positions = []
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                hand_positions.append((x, y))
    
    # 손 위치에 따라 스티어링 조정
    if hand_positions:
        hand_x, hand_y = hand_positions[0]
        # 손의 y 위치에 따라 스티어링 각도 조정
        steering_angle = np.interp(hand_y, [0, frame.shape[0]], [-30, 30])  # 화면 상단에서 하단으로 -30°에서 30° 사이로 조정
        
        draw_steering_wheel(frame, steering_angle)
        
        # 신호등 및 와이퍼 상태 업데이트
        signal_timer += 1 / 30  # 30 FPS
        if signal_timer > signal_interval:
            signal_left_on = not signal_left_on
            signal_right_on = not signal_right_on
            signal_timer = 0
        
        draw_signals(frame)
        draw_wiper(frame, wiper_on)
        
        # 속도 조정
        speed = min(max_speed, speed + acceleration / 30)  # 가속
        draw_speedometer(frame, speed, max_speed)
        
        # 비 효과 추가
        frame = add_rain_effect(frame, rain_intensity)
        
        # 기어 시프터 및 페달 그리기
        draw_gear_shifter(frame, gear)
        draw_pedals(frame, min(1, speed / max_speed), 0)  # 가속 페달, 브레이크 페달

    # 화면 표시
    cv2.imshow('Driving Simulator', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [3]:
!pip install mediapipe


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    cv2.imshow('Hand Tracking', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\project\deep\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [6]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def detect_hand_gesture(hand_landmarks):
    # 손바닥이 펼쳐졌는지 여부를 판단
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    palm_open = thumb_tip < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y and \
                index_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y and \
                middle_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y and \
                ring_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y and \
                pinky_tip < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y

    return palm_open

def main():
    cap = cv2.VideoCapture(0)

    with mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # 색상 채널을 BGR에서 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # 손 랜드마크 추출
            results = hands.process(image)

            # 색상 채널을 다시 RGB에서 BGR로 변환
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    if detect_hand_gesture(hand_landmarks):
                        cv2.putText(image, 'Palm Open - Car Moving', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        # 여기서 자동차가 움직이도록 제어하는 코드 추가
                    else:
                        cv2.putText(image, 'Fist - Car Stopped', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        # 여기서 자동차가 멈추도록 제어하는 코드 추가

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


In [7]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def detect_hand_gesture(hand_landmarks):
    # 손바닥이 펼쳐졌는지 여부를 판단
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    palm_open = thumb_tip < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y and \
                index_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y and \
                middle_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y and \
                ring_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y and \
                pinky_tip < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y

    return palm_open

def main():
    cap = cv2.VideoCapture(0)

    with mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # 색상 채널을 BGR에서 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # 손 랜드마크 추출
            results = hands.process(image)

            # 색상 채널을 다시 RGB에서 BGR로 변환
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    if detect_hand_gesture(hand_landmarks):
                        cv2.putText(image, 'Palm Open - Car Moving', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        # 여기서 자동차가 움직이도록 제어하는 코드 추가
                        # 예시: 자동차 시뮬레이터의 forward 함수 호출
                        # car_simulator.move_forward()
                    else:
                        cv2.putText(image, 'Fist - Car Stopped', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        # 여기서 자동차가 멈추도록 제어하는 코드 추가
                        # 예시: 자동차 시뮬레이터의 stop 함수 호출
                        # car_simulator.stop()

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


In [11]:
import cv2
import mediapipe as mp
import csv
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def detect_hand_gesture(hand_landmarks):
    # 손바닥이 펼쳐졌는지 여부를 판단
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    palm_open = thumb_tip < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y and \
                index_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y and \
                middle_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y and \
                ring_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y and \
                pinky_tip < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y

    return palm_open

def save_landmarks_to_csv(file_path, hand_landmarks, gesture):
    # CSV 파일이 없는 경우 헤더를 작성
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            header = ['gesture']
            for i in range(21):  # 21개의 손 랜드마크
                header += [f'x{i}', f'y{i}', f'z{i}']
            writer.writerow(header)

    # 랜드마크 좌표와 제스처를 파일에 저장
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        row = [gesture]
        for landmark in hand_landmarks.landmark:
            row += [landmark.x, landmark.y, landmark.z]
        writer.writerow(row)

def main():
    cap = cv2.VideoCapture(0)
    data_file = 'hand_gesture_data.csv'  # 데이터를 저장할 CSV 파일 경로

    with mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # 색상 채널을 BGR에서 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # 손 랜드마크 추출
            results = hands.process(image)

            # 색상 채널을 다시 RGB에서 BGR로 변환
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    if detect_hand_gesture(hand_landmarks):
                        cv2.putText(image, 'Palm Open', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        gesture = 'open'
                    else:
                        cv2.putText(image, 'Fist', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        gesture = 'fist'

                    # 손 랜드마크와 제스처를 CSV 파일에 저장
                    save_landmarks_to_csv(data_file, hand_landmarks, gesture)

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


In [12]:
import cv2
import mediapipe as mp
import numpy as np
import json

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def detect_hand_gesture(hand_landmarks):
    # 손바닥이 펼쳐졌는지 여부를 판단
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    palm_open = thumb_tip < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y and \
                index_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y and \
                middle_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y and \
                ring_finger_tip < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y and \
                pinky_tip < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y

    return palm_open

def main():
    cap = cv2.VideoCapture(0)

    results_json = {
        "gestures": []
    }

    with mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # 색상 채널을 BGR에서 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # 손 랜드마크 추출
            results = hands.process(image)

            # 색상 채널을 다시 RGB에서 BGR로 변환
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    gesture = "open" if detect_hand_gesture(hand_landmarks) else "fist"
                    results_json["gestures"].append({
                        "gesture": gesture,
                        "landmarks": [
                            {
                                "x": landmark.x,
                                "y": landmark.y,
                                "z": landmark.z
                            }
                            for landmark in hand_landmarks.landmark
                        ]
                    })

                    if gesture == "open":
                        cv2.putText(image, 'Palm Open - Car Moving', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        # 여기서 자동차가 움직이도록 제어하는 코드 추가
                    else:
                        cv2.putText(image, 'Fist - Car Stopped', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        # 여기서 자동차가 멈추도록 제어하는 코드 추가

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

    with open('hand_gestures.json', 'w') as f:
        json.dump(results_json, f, indent=4)

if __name__ == '__main__':
    main()
